# Market Basket Analysis Performance Assessment

### John Foster
### Department of Information Technology, Western Governors University
### D212: Data Mining II
### Professor Keiona Middleton
### June 12, 2023

### A1: PROPOSAL OF QUESTION

The research question I have decided to pursue using the Teleco Market Basket dataset is as follows:

“What are the 5 items that demonstrate the highest level of confidence with the most common item in the dataset?”

By identifying which combinations of items occur most frequently alongside the most common item in a given purchase, the organization can begin offering item discount incentives that statistically appeal to the greatest number of customers, hopefully mitigating the potential for customer churn across the greatest possible percent of the customer base by increasing customer satisfaction.

### A2: DEFINED GOAL

The primary goal of this analysis is twofold: finding the most commonly observed item purchased across the dataset, and then using that information to identify the 5 items that display the highest probability of appearing alongside that item in a purchase.

### B1: EXPLANATION OF MARKET BASKET

A market basket analysis searches for associations between items purchased by customers through the statistical evaluation of a large collection of recorded transactions, ultimately seeking to determine which items are most likely to be purchased together (Kamara, 2022).

Before the analysis can be performed, the dataset must first be sourced, cleaned, and consistently formatted. In our analysis, the transactions are formatted as lists of strings.

Next, an algorithm is applied to identify itemsets and their frequency. The algorithm used in our analysis is the Apriori algorithm. Essentially, this algorithm searches for combinations of items that appear frequently alongside one another iteratively, starting with “groups” of a single item, before incrementing group size by 1 at each pass and re-evaluating for frequency (referred to as “support”). THe Apriori algorithm maximizes its efficiency by pruning groups that do not appear often enough, thereby eliminating massive quantities of individual calculations that would otherwise be performed moving forward. The selection criteria for pruning is generated as input by the user in the form of a predefined minimum threshold (in our analysis, this threshold will be .03). The threshold also acts as the stopping criterion for the algorithm when all new potential itemsets evaluated in an iteration fail to meet the threshold.

Once the list of itemsets is generated, the algorithm calculates association rules between items in the dataset, which explicitly describe the nature of the relationships between the given items. Once these rules are generate, we will first identify the item that appears most frequently in the database (demonstrating the highest level of ‘support’ according to the algorithm), then generate a dataframe containing the rules associated with that item. We will then select the 5 rules with the highest degree of ‘confidence’ (the probability of a ‘consequent’ item appearing based on the presence of a given ‘antecedent’ item). This will effectively satisfy our research question.

### B2: TRANSACTION EXAMPLE

The following code returns an example of a customer transaction evaluated in our analysis:

In [98]:
#Import the libraries that we need for our analysis.
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder

In [99]:
#Import the churn dataset from the source csv file and make a dataframe out of it.
raw_df = pd.read_csv('teleco_market_basket.csv')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

#Make a copy of the imported dataframe in case we would like to inspect or manipulate the raw data. 
df = raw_df.copy()

In [100]:
print(df.head(2).tail(1))

                         Item01     Item02     Item03                      Item04                     Item05        Item06                        Item07                               Item08                         Item09                                   Item10                        Item11                     Item12                        Item13                       Item14                          Item15                          Item16                 Item17                    Item18                                Item19                               Item20
1  Logitech M510 Wireless mouse  HP 63 Ink  HP 65 ink  nonda USB C to USB Adapter  10ft iPHone Charger Cable  HP 902XL ink  Creative Pebble 2.0 Speakers  Cleaning Gel Universal Dust Cleaner  Micro Center 32GB Memory card  YUNSONG 3pack 6ft Nylon Lightning Cable  TopMate C5 Laptop Cooler pad  Apple USB-C Charger cable  HyperX Cloud Stinger Headset  TONOR USB Gaming Microphone  Dust-Off Compressed Gas 2 pack  3A USB Type C Cable 3 p

### B3: MARKET BASKET ASSUMPTION

One assumption of a market basket analysis is that there is a specified minimum support threshold identified for use in the Apriori algorithm. This threshold acts as the filtering criteria that allows the algorithm to prune itemsets that do not demonstrate an adequate amount of support within the dataset (thereby maximizing computational efficiency), and eventually acts as the stopping criterion when all new potential itemsets fail to meet it.

### C1: TRANSFORMING THE DATASET

There are several steps necessary to transform the dataset and prepare it for processing in a market basket analysis. First, upon examining the imported source .csv file in a dataframe, we can observe that every other row in the dataset is a null row. We will use the .dropna(how=’all’) method to quickly drop those rows. When we evaluate the individual columns for nullity using the .info function, we can see that there is more and more nullity in each column as the item number increases, as there are more transactions in the dataset with fewer items.

In order to account for this nullity, we will collect each row as a list, with each column’s value stored as a string, but only if the value in the given cell is not null. These lists will be stored in a master list so that we can iterate through them cleanly using our TransactionEncoder method, which converts the item values of each transaction to boolean values, thereby formatting it correctly for the Apriori algorithm. This concludes the necessary preprocessing of our dataset.

The code to execute the preprocessing steps can be found below.

In [101]:
print(df.head())

                                  Item01                            Item02        Item03                      Item04                     Item05        Item06                        Item07                               Item08                         Item09                                   Item10                        Item11                     Item12                        Item13                       Item14                          Item15                          Item16                 Item17                    Item18                                Item19                               Item20
0                                    NaN                               NaN           NaN                         NaN                        NaN           NaN                           NaN                                  NaN                            NaN                                      NaN                           NaN                        NaN                           NaN                

In [102]:
#Drop the null rows
df.dropna(how='all', inplace=True)

In [103]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7501 entries, 1 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 1.2+ MB
None


In [104]:
#Create a master list for each row's list of string values
master_list = []
#Make our nested for loop to create and populate our lists
for index in range(len(df)):
    row_list = []
    for value in range(len(df.columns)):
        # Add the value from the cell as long as it isn't a null value
        if pd.isnull(df.iloc[index, value]) is False:
            row_list.append(str(df.values[index, value]))
    master_list.append(row_list)

In [105]:
#Encode our transactions to binary by converting them to an array and then making a dataframe out of it
TransEncoder = TransactionEncoder()
encoded_array = TransEncoder.fit_transform(master_list)
encoded_df = pd.DataFrame(encoded_array, columns=TransEncoder.columns_)
print(encoded_df)

#Export the dataframe to a .csv file
encoded_df.to_csv('encoded_df.csv', index=False)

      10ft iPHone Charger Cable  10ft iPHone Charger Cable 2 Pack  3 pack Nylon Braided Lightning Cable  3A USB Type C Cable 3 pack 6FT  5pack Nylon Braided USB C cables  ARRIS SURFboard SB8200 Cable Modem  Anker 2-in-1 USB Card Reader  Anker 4-port USB hub  Anker USB C to HDMI Adapter  Apple Lightning to Digital AV Adapter  Apple Lightning to USB cable  Apple Magic Mouse 2  Apple Pencil  Apple Pencil 2nd Gen  Apple Power Adapter Extension Cable  Apple USB-C Charger cable  AutoFocus 1080p Webcam  BENGOO G90000 headset  Blue Light Blocking Glasses  Blue Light Blocking Glasses 2pack  Brother Genuine High Yield Toner Cartridge  Cat 6 Ethernet Cable 50ft  Cat8 Ethernet Cable  CicTsing MM057 2.4G Wireless Mouse  Cleaning Gel Universal Dust Cleaner  Creative Pebble 2.0 Speakers  DisplayPort ot HDMI adapter  Dust-Off Compressed Gas  Dust-Off Compressed Gas 2 pack  FEEL2NICE 5 pack 10ft Lighning cable  FEIYOLD Blue light Blocking Glasses  Falcon Dust Off Compressed Gas  \
0                    

### C2: CODE EXECUTION

For continuity and ease of presentation, the code snippets used in this analysis have been included in the rubric sections associated with them, and can be run together at once in this notebook file by clicking the “Run All” button at the top of the window.

### C3: ASSOCIATION RULES TABLE

The following code returns our association rules table using lift as the primary metric, and includes values for support, lift, and confidence for each rule:

In [106]:
#We are now ready to run the Apriori algorithm and calculate our support values
f_sets = apriori(encoded_df, min_support = 0.03, use_colnames = True)

# Return the item with the highest support in the dataset and store it for later
max_index = f_sets['support'].idxmax()
max_value = f_sets.loc[max_index, 'itemsets']

print(max_value)

frozenset({'Dust-Off Compressed Gas 2 pack'})


#### Attribution: 

Apriori function and associated methods used in this assessment are adapted from the following source: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/ (Raschka, 2014-2023)

In [107]:
#Filter association rules so only rules with a positive lift are returned, then order by lift value
#This will return our association rules table
rules = association_rules(f_sets, metric = 'lift', min_threshold = 1.0)
print(rules)

                               antecedents                             consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction  zhangs_metric
0         (Dust-Off Compressed Gas 2 pack)                          (Apple Pencil)            0.238368            0.179709  0.050927    0.213647  1.188845  0.008090    1.043158       0.208562
1                           (Apple Pencil)        (Dust-Off Compressed Gas 2 pack)            0.179709            0.238368  0.050927    0.283383  1.188845  0.008090    1.062815       0.193648
2                              (HP 61 ink)                          (Apple Pencil)            0.163845            0.179709  0.033196    0.202604  1.127397  0.003751    1.028711       0.135143
3                           (Apple Pencil)                             (HP 61 ink)            0.179709            0.163845  0.033196    0.184718  1.127397  0.003751    1.025603       0.137757
4          (Screen Mom Screen Cleaner ki

### C4: TOP THREE RULES

The top three association rules in our table sorted by lift are returned by executing the following code:

In [108]:
lift_rules = rules.sort_values(by='lift', ascending= False)
top_3_rules = lift_rules[:3]
print(top_3_rules)

                           antecedents                         consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction  zhangs_metric
29  (VIVO Dual LCD Monitor Desk mount)           (SanDisk Ultra 64GB card)            0.174110            0.098254  0.039195    0.225115  2.291162  0.022088    1.163716       0.682343
28           (SanDisk Ultra 64GB card)  (VIVO Dual LCD Monitor Desk mount)            0.098254            0.174110  0.039195    0.398915  2.291162  0.022088    1.373997       0.624943
15    (Dust-Off Compressed Gas 2 pack)           (SanDisk Ultra 64GB card)            0.238368            0.098254  0.040928    0.171700  1.747522  0.017507    1.088672       0.561638


### D1: SIGNIFICANCE OF SUPPORT, LIFT, AND CONFIDENCE SUMMARY

In our analysis, we returned the top three association rules, filtered by lift. The first two rules feature the same two items, with the antecedents and consequents reversed. We will now explain the support, lift, and confidence values of our top three association rules.

We can define support as the percentage of observations that demonstrate the given rule agains the total number of observations. As we can see in our code output, our top three rules demonstrate support levels of .039195, .039195, and .040928.

Life indicates whether or not a statistically significant relationship exists between the presence of two items in a given purchase order.  A lift value above 1 indicates a positive relationship between the items, a value below 1 indicates a negative relationship (meaning the presence of the antecedent item makes a transaction less likely to contain the consequent item), and a value at or very near 1 indicates no significant relationship. In our results, our top three rules demonstrate lift values of 2.291162, 2.291162, and 1.747522.

Confidence refers to the frequency of transactions in which the consequent item appears when the antecedent item is present, making it a functional measure of relative consequent support within the antecedent-supported subset of the total dataset. In our top three rules, the confidence levels are .225115, .398915, and .171700.

### Now we will execute the code to conclude our own analysis and answer our research question.

In [109]:
ant_df = rules[rules['antecedents'] == {'Dust-Off Compressed Gas 2 pack'}]
final_df = ant_df.sort_values(by='confidence', ascending= False)
print(final_df.head())

                         antecedents                         consequents  antecedent support  consequent support   support  confidence      lift  leverage  conviction  zhangs_metric
21  (Dust-Off Compressed Gas 2 pack)  (VIVO Dual LCD Monitor Desk mount)            0.238368            0.174110  0.059725    0.250559  1.439085  0.018223    1.102008       0.400606
10  (Dust-Off Compressed Gas 2 pack)                         (HP 61 ink)            0.238368            0.163845  0.052660    0.220917  1.348332  0.013604    1.073256       0.339197
0   (Dust-Off Compressed Gas 2 pack)                      (Apple Pencil)            0.238368            0.179709  0.050927    0.213647  1.188845  0.008090    1.043158       0.208562
17  (Dust-Off Compressed Gas 2 pack)     (Screen Mom Screen Cleaner kit)            0.238368            0.129583  0.047994    0.201342  1.553774  0.017105    1.089850       0.467950
15  (Dust-Off Compressed Gas 2 pack)           (SanDisk Ultra 64GB card)            0.2383

### D2: PRACTICAL SIGNIFICANCE OF FINDINGS

In answering our research question, we found that the item that appeared most frequently in transactions was ‘Dust-Off Compressed Gas 2 pack.’ When association rules were selected using this item as the antecedent, the top 5 rules returned when sorted by confidence were ‘VIVO dual LCD Monitor Desk mount,’ ‘HP 61 ink,’ ‘Apple Pencil,’ ‘Screen Mom Screen Cleaner kit,’ and ‘SanDisk Ultra 64 GB card.’ The rules associated with these relationships exhibited support levels ranging from .040928 to .059725, lift from 1.188845 to 1.747522, and confidence levels from .171700 to .250559

None of these results are terribly extreme in nature, but they do demonstrate the existence of statistically significant relationships between the presence of these items in a given order. They indicate that these rules occur in roughly 4-5% of total transactions, have a moderate positive strength in their association, and have a 17-25% to be demonstrated when ‘Dust-Off Compressed Gas 2 pack’ is purchased.

### D3: COURSE OF ACTION

Based on the results of this analysis, I would recommend that modest discounts are considered as customer incentives for these items in particular to bolster customer satisfaction and loyalty. THe discounts could be structured to apply only when 2 or more of the associated items are purchased to maximize impulse buying, and further market basket analyses could be conducted to explore rules related to other frequently-occurring items, following the same formula of incentivizing common combinations of items alongside items that demonstrate the highest levels of antecedent support within the greater transaction dataset.

### E: PANOPTO RECORDING

A brief demonstration video of this code, including a description of the software environment used to develop it, has been recorded via Panopto and submitted for evaluation in conjunction with this Jupyter notebook per instructions in the WGU submission portal.

The software used in the production of this code is as follows:

Operating System: Windows 11 Home<br>
Code Editor and Integrated Development Environment: Microsoft Visual Studio Code<br>
Browser used for research and submission: Google Chrome<br>
Spreadsheet editor: Microsoft Excel

### F: WEB SOURCES

Raschka, S. (2014-2023). Apriori: Frequent Itemsets Via the Apriori Algorithm. https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

### G: SOURCES

Kamara, K. (2022). Market Basket Analysis in Theory. Western Governors University, College of Information Technology. https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=9541a29b-2f14-4c5d-9d86-af030005bcf6.